In [29]:
import requests
import json
import sqlite3
import os
from dotenv import load_dotenv
import os
from openai import OpenAI
from pathlib import Path

In [ ]:
def get_client():
    """
    Get the client object

    Returns:
        client (?): The client object for OpenAI
    """
    load_dotenv()
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    return client

In [ ]:
import openai


def generate_sql_query(openai_client, user_query):
    """
    Generates an SQL-query based on the user's query in Norwegian.

    Parameters:
        openai_client (OpenAI): An authenticated OpenAI API client object (e.g., openai).
        user_query (str): The user's query in Norwegian, e.g. "Vis alle driftsmeldinger der meldingstype er 'Alarm' og stasjon er 'Kobbervik'."
    
    Returns:
        str: A string with the SQL-query. 
    """

    db_schema_description = """
    Du har en SQLite-database med én tabell kalt 'driftsmeldinger'.
    Tabellen har følgende kolonner:
      - id INTEGER PRIMARY KEY AUTOINCREMENT
      - tid DATETIME NOT NULL
      - hendelse VARCHAR(255) NOT NULL
      - meldingstype VARCHAR(50) NOT NULL
      - stasjon VARCHAR(50) NOT NULL
    """

    system_message = f"""
    Du er en hjelpsom assistent som genererer gyldige SQL-spørringer (for SQLite) basert på en brukerforespørsel.
    Brukeren jobber med følgende databaseskjema:
    {db_schema_description}

    Når du genererer SQL-spørringen:
      - Bruk korrekt SQLite-syntaks.
      - Returner kun spørringen, uten ekstra forklaringer.
    """

    user_message = f"""
    {user_query}
    """

    response = openai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_message},
        ],
        temperature=0
    )

    sql_query = response.choices[0].message.content.strip()
    return sql_query

In [26]:
def run_sql_query(db_path, sql_query):
    """
    Executes the provided SQL query against the SQLite database located at db_path.
    Returns all fetched rows.

    :param db_path: Path to the SQLite database file (e.g. 'data/status_updates.db').
    :param sql_query: The SQL query to be executed.
    :return: A list of tuples representing the rows returned by the query.
    """
    try:
        with sqlite3.connect(db_path) as conn:
            cursor = conn.cursor()
            cursor.execute(sql_query)
            results = cursor.fetchall()
        return results
    except sqlite3.Error as e:
        print(f"SQLite error: {e}")
        return []

In [ ]:
#sql-parser


In [24]:
user_query = "Hendelser med meldingstype 'Alarm' og stasjon 'Kobbervik'."
openai_client = get_client()
sql_query = generate_sql_query(openai_client, user_query)

In [31]:
database_path = "../data/status_updates.db"
print(f"Database path: {database_path}")
run_sql_query(database_path, sql_query)

Database path: ../data/status_updates.db


[(11,
  '2024-12-31 21:02:11',
  'Kobbervik G1 Feil på sensor',
  'Alarm',
  'Kobbervik'),
 (16,
  '2025-01-01 00:12:12',
  'Kobbervik G2 Overoppheting registrert',
  'Alarm',
  'Kobbervik')]

Thoughts: 
